# ***ECO482 Group Project: An American Disaster Story: Predicting the Economic Damage Inflicted by Tornados​***

In [44]:
#mounting/loading google drive (working on google colab)
from google.colab import drive
drive.mount('/content/drive')

#loading useful packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# Step 2: Define the folder path (ensure correctness)
folder_path = "/content/drive/My Drive/GDToT/ECO482_Project/Data"

# Step 3: Use glob to get all CSV file paths
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))  # Assumes all files end in .csv

# Step 4: Load and concatenate all CSVs
df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
storm_data = pd.concat(df_list, ignore_index=True)  # Combine into one DataFrame


# Optionally, show first few rows
storm_data.head()

<ipython-input-58-ad6e382d2ef2>:8: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-58-ad6e382d2ef2>:8: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-58-ad6e382d2ef2>:8: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-58-ad6e382d2ef2>:8: DtypeWarning: Columns (26,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(file) for file in csv_files]  # Read each CSV into a list
<ipython-input-58-ad6e382d2ef2>:8: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on impor

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,198904,3,1542,198904,3,1542,NaN,9990856,ARKANSAS,5.0,...,0.0,NaN,NaN,36.33,-93.57,NaN,NaN,NaN,NaN,PUB
1,198907,10,1430,198907,10,1430,NaN,9985066,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.72,-73.48,NaN,NaN,NaN,NaN,PUB
2,198907,10,1440,198907,10,1440,NaN,9985067,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.83,-73.33,41.72,-73.23,NaN,NaN,PUB
3,198907,10,1515,198907,10,1515,NaN,9985068,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.60,-73.12,41.57,-73.08,NaN,NaN,PUB
4,198907,10,1515,198907,10,1515,NaN,9985069,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.68,-73.20,NaN,NaN,NaN,NaN,PUB


In [59]:
storm_data

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,198904,3,1542,198904,3,1542,NaN,9990856,ARKANSAS,5.0,...,0.0,NaN,NaN,36.3300,-93.5700,NaN,NaN,NaN,NaN,PUB
1,198907,10,1430,198907,10,1430,NaN,9985066,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.7200,-73.4800,NaN,NaN,NaN,NaN,PUB
2,198907,10,1440,198907,10,1440,NaN,9985067,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.8300,-73.3300,41.7200,-73.2300,NaN,NaN,PUB
3,198907,10,1515,198907,10,1515,NaN,9985068,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.6000,-73.1200,41.5700,-73.0800,NaN,NaN,PUB
4,198907,10,1515,198907,10,1515,NaN,9985069,CONNECTICUT,9.0,...,0.0,NaN,NaN,41.6800,-73.2000,NaN,NaN,NaN,NaN,PUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862526,202112,11,532,202112,11,532,163794.0,993981,ALABAMA,1.0,...,1.0,WNW,PETERSVILLE,34.8596,-87.7011,34.8596,-87.7011,A quasi-linear convective system (QLCS) moved ...,A tree was knocked down onto a residence at 45...,CSV
1862527,202112,11,532,202112,11,532,163794.0,993982,ALABAMA,1.0,...,1.0,NW,ST FLORIAN,34.8751,-87.6263,34.8751,-87.6263,A quasi-linear convective system (QLCS) moved ...,Trees were knocked down on CR 30 at the Cathol...,CSV
1862528,202112,11,533,202112,11,533,163794.0,993986,ALABAMA,1.0,...,3.0,ENE,ST FLORIAN,34.8900,-87.5800,34.8900,-87.5800,A quasi-linear convective system (QLCS) moved ...,Trees and power lines were knocked down at the...,CSV
1862529,202112,11,534,202112,11,534,163794.0,993987,ALABAMA,1.0,...,4.0,WSW,COLBERT HGTS,34.6438,-87.7650,34.6438,-87.7650,A quasi-linear convective system (QLCS) moved ...,A tree was knocked down on Frankfurt Road.,CSV


In [62]:
storm_data.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [74]:
storm_data["DURATION"] = (storm_data["END_YEARMONTH"] - storm_data["BEGIN_YEARMONTH"])*30 + storm_data["END_DAY"] - storm_data["BEGIN_DAY"]
storm_data['COUNTY/PARISH'] = (storm_data['CZ_TYPE'] == "C").astype(int)
storm_data['MARINE_ZONE'] = (storm_data['CZ_TYPE'] == "M").astype(int)
storm_data['NWS_PUBLIC_FORECAST_ZONE'] = (storm_data['CZ_TYPE'] == "Z").astype(int)

In [77]:
storm_data[['DURATION', 'TOR_F_SCALE', 'TOR_WIDTH', 'TOR_LENGTH', 'EVENT_TYPE', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'COUNTY/PARISH', 'MARINE_ZONE', 'NWS_PUBLIC_FORECAST_ZONE' , 'STATE', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT']].describe()

,DURATION,TOR_WIDTH,TOR_LENGTH,BEGIN_YEARMONTH,END_YEARMONTH,COUNTY/PARISH,MARINE_ZONE,NWS_PUBLIC_FORECAST_ZONE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT
count,1.862531e+06,197939.000000,197939.000000,1.862531e+06,1.862531e+06,1.862531e+06,1.862531e+06,1.862531e+06,1.862531e+06,1.862531e+06,1.862531e+06,1.862531e+06
mean,1.578231e+00,39.588202,0.864149,2.009119e+05,2.009119e+05,6.299417e-01,4.033758e-03,3.660245e-01,5.809836e-02,8.264560e-03,1.057486e-02,2.247479e-03
std,1.530923e+01,145.939898,8.912679,1.003089e+03,1.003088e+03,4.828201e-01,6.338367e-02,4.817164e-01,2.621280e+00,1.866846e+00,5.704095e-01,9.445378e-02
min,0.000000e+00,0.000000,0.000000,1.980010e+05,1.980010e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000,0.000000,2.002050e+05,2.002050e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000,0.000000,2.010060e+05,2.010060e+05,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,20.000000,0.100000,2.018010e+05,2.018010e+05,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.002000e+03,4576.000000,2315.000000,2.024110e+05,2.024110e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.150000e+03,2.400000e+03,6.380000e+02,5.600000e+01
